
</h1> <h1 style='direction:rtl; font-family: "B Lotus";'> تمرین دوم - پردازش شعرهای با لهجه‌ی شیرازی

    
  

<h2 style='direction:rtl;font-family: "B Lotus";'> داده‌ی ورودی </h2> 

<p style='direction:rtl;font-family: "B Lotus";' >
</p>


In [1]:
path_dir = "./data"


In [2]:
! pip install pandas
! pip install hazm
! pip install transformers
! pip install torch

  Using cached torch-1.11.0-cp38-cp38-manylinux1_x86_64.whl (750.6 MB)


In [3]:
import random 
import numpy as np
import nltk
import pandas as pd
import codecs
import tqdm
import os
import re
import glob
from __future__ import unicode_literals
from hazm import *
from nltk import FreqDist
import itertools
import pandas as pd

In [4]:
class Cleaner:
    def __init__(self):
        self.normalizer = Normalizer()
        # self.stop_words = [self.normalizer.normalize(x.strip()) for x in codecs.open('data/stopwords.txt','r','utf-8').readlines()]
        self.file_range = None


    def read_poems(self, file_patterns: str, file_range: tuple, normalize=False):  
        file_range = range(*file_range) if file_range is not None else None 
        file_names = glob.glob(file_patterns)
        mesra_collection = []
        for file_name in file_names:
            if normalize:
                mesra_collection += [self.normalizer.normalize(x) for x in codecs.open(file_name,'rU','utf-8').readlines()]
            else:
                mesra_collection += codecs.open(file_name,'rU','utf-8').readlines()[2:]
        return mesra_collection
    

    def read_documents(self):
        poems = self.read_poems(self.file_pattern, self.file_range)
        cleaned = self.clean_data(poems)
        return ' \n '.join(cleaned)
    

In [13]:
path=f"{path_dir}/ferdousi.txt"
cleaner = Cleaner()
mesras = cleaner.read_poems(file_patterns=path, file_range=None, normalize=False)
print(len(mesras))

99217


In [6]:
from transformers import AutoConfig, AutoTokenizer, AutoModel, TFAutoModel

# v3.0
model_name_or_path = "HooshvareLab/bert-fa-zwnj-base"
config = AutoConfig.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# model = TFAutoModel.from_pretrained(model_name_or_path)  For TF
model = AutoModel.from_pretrained(model_name_or_path)

text = "ما در هوش‌واره معتقدیم با انتقال صحیح دانش و آگاهی، همه افراد میتوانند از ابزارهای هوشمند استفاده کنند. شعار ما هوش مصنوعی برای همه است."
tokenizer.tokenize(text)


Downloading: 100%|██████████| 452M/452M [03:31<00:00, 2.23MB/s]   
Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-b

['ما',
 'در',
 'هوش',
 '[ZWNJ]',
 'واره',
 'معتقدیم',
 'با',
 'انتقال',
 'صحیح',
 'دانش',
 'و',
 'آ',
 '##گاهی',
 '،',
 'همه',
 'افراد',
 'میتوانند',
 'از',
 'ابزارهای',
 'هوشمند',
 'استفاده',
 'کنند',
 '.',
 'شعار',
 'ما',
 'هوش',
 'مصنوعی',
 'برای',
 'همه',
 'است',
 '.']

In [27]:
from transformers import (
    AutoTokenizer,
    TextDataset,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
    AutoModelWithLMHead,
)


def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
        tokenizer=tokenizer, file_path=train_path, block_size=256
    )

    test_dataset = TextDataset(tokenizer=tokenizer, file_path=test_path, block_size=256)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    )

    return train_dataset, test_dataset, data_collator


# Freezing the lower layers increases the training speed and reduces the memory requirement.
# Depending on your task you may want to freeze all layers and train addition layers that you are adding to the model
# or unfreeze as many layers that you can affort training with a reasonable batchsize.
def freeze_lower_layers():
    for param in model.base_model.parameters():
        param.requires_grad = False

    for param in (
        model.base_model.h[23].parameters() or model.base_model.h[22].parameters()
    ):
        param.requires_grad = True


# load model
model = AutoModelWithLMHead.from_pretrained("bolbolzaban/gpt2-persian")

# freeze lower layers and only train top layers
freeze_lower_layers()

# load dataset
tokenizer = AutoTokenizer.from_pretrained("bolbolzaban/gpt2-persian")
train_dataset, test_dataset, data_collator = load_dataset(
    "./data/ferdousi.txt", "./data/test.txt", tokenizer
)
print(type(train_dataset))


/home/sahel/university/NLP/.venv/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:921: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
loading configuration file https://huggingface.co/bolbolzaban/gpt2-persian/resolve/main/config.json from cache at /home/sahel/.cache/huggingface/transformers/55af8b32edc9c441878906a22de9b0c79dce98654714626407f7ea3ee7fc7349.1f2d676354b04ed818190819fee0d7d2deb97cfe456b804edfea24e6a8053fd5
Model config GPT2Config {
  "_name_or_path": "bolbolzaban/gpt2-persian",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_dropout": 0.1,
  "attn_pdrop": 0.1,
  "beta1": 0.9,
  "beta2": 0.98,
  "bos_token_id": 8,
  "data_path": "",
  "embd_pdrop": 0.1,
  "embed_dropout": 0.

<class 'transformers.data.datasets.language_modeling.TextDataset'>


In [28]:
from sklearn.model_selection import train_test_split
train_texts, val_texts = train_test_split(train_dataset)

In [21]:
print(type(train_texts))

<class 'list'>


In [23]:
print(type(val_texts))
print(type(train_dataset))

<class 'list'>
<class 'transformers.data.datasets.language_modeling.TextDataset'>


In [29]:
print(len(train_texts))
print(len(val_texts))
print(len(train_dataset))

1895
632
2527


In [30]:
# train
training_args = TrainingArguments(
    output_dir="./model",
    overwrite_output_dir=True,
    num_train_epochs=5,
    # Set the batch size to a maximum value that could fit into GPU memory,
    # for example 12 is the largest batch size that could work on a 6gb GPU when training the last to layers
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    eval_steps=1000,
    save_steps=1000,
    warmup_steps=500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_texts,
    eval_dataset=val_texts,
)


trainer.train()

# save
trainer.save_model()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 1895
  Num Epochs = 5
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 1
  Total optimization steps = 790








































































































































































































































































































































































































































































{'loss': 5.2555, 'learning_rate': 5e-05, 'epoch': 3.16}






































































































































































































































































































Training completed. Do not forget to share your model on huggingface.co/models =)



                                                  

                                                      
100%|██████████| 790/790 [2:16:06<00:00, 10.34s/it]/s]
Saving model checkpoint to ./model
Configuration saved in ./model/config.json


{'train_runtime': 8166.6446, 'train_samples_per_second': 1.16, 'train_steps_per_second': 0.097, 'train_loss': 5.17218388424644, 'epoch': 5.0}


Model weights saved in ./model/pytorch_model.bin
